In [1]:
!pip install pdf2image

In [ ]:
!conda install -c conda-forge poppler

In [1]:
!pip install openpyxl

In [1]:
import numpy as np
import json
import glob

# Extract image
import pytesseract
import cv2
from PIL import Image

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel

# spacy
import spacy
from nltk.corpus import stopwords

# lda visual
import pyLDAvis
import pyLDAvis.gensim

# PDF 
from pdf2image import convert_from_path

# Clean Data
from bs4 import BeautifulSoup # 4.6.3
import re

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Step 1, OCR
## Extrak Text From PDF File

In [2]:
# Path ke file PDF
pdf_file = "assets/jurnal.pdf"
result = []

# Fungsi untuk mengekstrak teks dari file PDF dan menampilkan contour
def extract_text_and_show_contours(pdf_path):
    
    # Ubah file PDF menjadi gambar (gambar per halaman)
    
    images = []
    
    try:
        images = convert_pdf_to_images(pdf_path)
    except Exception as e:
        print(f"Error: {e}")

    # Loop melalui setiap gambar (halaman) dan ekstrak teks serta tampilkan contour
    for i, image in enumerate(images):
        text = show_text_contours(image, i)
        print(f"Hasil ekstraksi teks dari halaman {i + 1}")
        
#         show_text_contours(image)

# Fungsi untuk mengubah file PDF menjadi daftar gambar
def convert_pdf_to_images(pdf_path):
    images = []

    # Fungsi untuk mengonversi PDF menjadi gambar
    pdf_images = convert_from_path(pdf_path)

    # Simpan setiap gambar sebagai file gambar terpisah
    for i, image in enumerate(pdf_images):
        images.append(f"convert_to_img/page_{i + 1}.png")
        image.save(f'convert_to_img/page_{i + 1}.png', 'PNG')
                
    return images

# Fungsi untuk mengekstrak teks dari gambar menggunakan pytesseract
def extract_text_from_image(image):

    ocr_result = pytesseract.image_to_string(image)
    ocr_result = ocr_result.split("\n")

    for line in ocr_result:

        result.append(line)
                
    return ocr_result

# Fungsi untuk menampilkan contour dari gambar
def show_text_contours(image, index_img):
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 50))
    dilate = cv2.dilate(thresh, kernal, iterations=1)
    
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])
    
    if_agains = 0
        
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)

        # if statement berikut untuk mengambil bagian isi dari image, jika tidak maka semua kata akan di contours

        if h > 200 and w > 250:
            roi = image[y:y+h, x:x+w]
            
            if if_agains > 0:
                cv2.imwrite(f"temp/jurnal_countor_{index_img}_{if_agains}.png", roi)
                cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
            else:
                cv2.imwrite(f"temp/jurnal_countor_{index_img}.png", roi)
                cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
            
            extract_text_from_image(roi)
            if_agains += 1
                
    cv2.imwrite(f"boxes/sample_boxes_{index_img}.png", image)


In [3]:
extract_text_and_show_contours(pdf_file)

Hasil ekstraksi teks dari halaman 1
Hasil ekstraksi teks dari halaman 2
Hasil ekstraksi teks dari halaman 3
Hasil ekstraksi teks dari halaman 4
Hasil ekstraksi teks dari halaman 5
Hasil ekstraksi teks dari halaman 6


In [4]:
result

['   ',
 ' ',
 '  ',
 '   ',
 '',
 'san Artikel',
 'Teknologi Na',
 '',
 'PENULIS”, DST',
 '',
 'titusi asal penulis 1)',
 'titusi asal penulis 2)',
 '',
 ' ',
 '\x0c',
 ' ',
 '',
 'NAMA PENULIS?, NAMA PENULIS2, DST',
 '',
 '    ',
 '',
 '1. Afiliasi Penulis 1 (ingtitusi asal penulis 1)',
 '2. Afiliasi Penulis 2 (institusi asal penulis 2)',
 '3. d',
 '',
 'Email (penulis 1 saja): o-bama@itenas.ac.id',
 '',
 'ABSTRAK',
 '',
 'Jurnal Online Institut Teknologi Nasional Bandung terdiri dari 13 jurnal yang',
 'memiliki fokus pada 13 bidang kajian ilmiah. Untuk memudahkan proses',
 'penyuntingan, para penulis disarankan untuk mengikuti petunjuk penulisan',
 'artikel ini secara keseluruhan. Format dan syle yang terdapat dalam file ini sudah',
 'sesuai dengan spesifikasi yang tertulis dalam petunjuk penulisan, sehingga file ini',
 'dapat digunakan sebagai template. Jumlah halaman penulisan adalah antara 8',
 'sampai 12 halaman, termasuk di dalamnya gambar, tabel, daftar rujukan, dan',
 'abstra

# Step 2, Clean Data

In [5]:
def clean_data(data):
    data = BeautifulSoup(data, "lxml").get_text()
    
    # Delete the @
    data = re.sub(r"@[A-Za-z0-9]+", ' ', data)
    
    # Delete URL links
    data = re.sub(r"https?://[A-Za-z0-9./]+", ' ', data)
    
    # Just keep letters and important punctuation
    data = re.sub(r"[^a-zA-Z.!?']", ' ', data)
    
    # Just keep letters and important punctuation
    # data = re.sub('(?i)[^0-9a-z.!?]', '', data)
    
#     data = re.sub(r'[.,;]', '', data)
    
    # Remove additional spaces
    data = re.sub(r" +", ' ', data)
    
    # Remove spaces at the beginning and end of text
    data = data.strip()
    
    return data

In [6]:
data_clean = []


for item in result:
    item = item.strip()
    data = clean_data(item)
        
    if len(data) > 4:
        data_clean.append(data)

print(data_clean)

['san Artikel', 'Teknologi Na', 'PENULIS DST', 'titusi asal penulis', 'titusi asal penulis', 'NAMA PENULIS? NAMA PENULIS DST', '. Afiliasi Penulis ingtitusi asal penulis', '. Afiliasi Penulis institusi asal penulis', 'Email penulis saja o bama .ac.id', 'ABSTRAK', 'Jurnal Online Institut Teknologi Nasional Bandung terdiri dari jurnal yang', 'memiliki fokus pada bidang kajian ilmiah. Untuk memudahkan proses', 'penyuntingan para penulis disarankan untuk mengikuti petunjuk penulisan', 'artikel ini secara keseluruhan. Format dan syle yang terdapat dalam file ini sudah', 'sesuai dengan spesifikasi yang tertulis dalam petunjuk penulisan sehingga file ini', 'dapat digunakan sebagai template. Jumlah halaman penulisan adalah antara', 'sampai halaman termasuk di dalamnya gambar tabel daftar rujukan dan', 'abstrak dalam Bahasa Indonesia dan Bahasa Inggris. Artikel diserahkan kepada', 'pengelola Jurnal Online berupa softcopy dalam format MS Word .doc atau', '.docx yang diunggah melalui setelah penu

C:\Users\Raqwan\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Raqwan\anaconda3\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


# Step 3, Preprocessing

In [7]:
# allowed_postags=["NOUN", "ADJ", "VERB", "ADV", "PROPN", "WORK_OF_ART", "ORG", "GPE"]
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV", "PROPN"]):
    
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    
    for text in texts:
        doc = nlp(text)
        new_text = []
        
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
                
        final = " ".join(new_text)
        texts_out.append(final)
        
    return (texts_out)


lemmatized_texts = lemmatization(data_clean)
lemmatized_texts

['san Artikel',
 'Teknologi Na',
 'penulis dst',
 'titusi asal penuli',
 'titusi asal penuli',
 'NAMA PENULIS NAMA PENULIS DST',
 'Afiliasi Penulis ingtitusi asal penuli',
 'Afiliasi Penulis asal penuli',
 'email penuli saja o bama',
 'ABSTRAK',
 'Jurnal Online Institut Teknologi Nasional Bandung terdiri dari jurnal yang',
 'memiliki fokus pada bidang kajian ilmiah Untuk memudahkan prose',
 'penyuntingan para penuli disarankan untuk mengikuti petunjuk penulisan',
 'artikel ini secara keseluruhan Format dan syle yang terdapat dalam file ini sudah',
 'sesuai dengan spesifikasi yang tertulis dalam petunjuk penulisan sehingga file ini',
 'dapat digunakan sebagai template Jumlah halaman penulisan adalah antara',
 'sampai halaman termasuk di dalamnya gambar tabel daftar rujukan dan',
 'abstrak dalam Bahasa Indonesia dan Bahasa Inggris artikel kepada',
 'pengelola Jurnal Online berupa softcopy dalam format MS Word atau',
 'yang diunggah melalui setelah penuli',
 'melakukan proses registrasi a

In [8]:
def gen_words(texts):
    final = []
    
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
        
    return (final)

data_words = gen_words(lemmatized_texts)
data_words

[['san', 'artikel'],
 ['teknologi', 'na'],
 ['penulis', 'dst'],
 ['titusi', 'asal', 'penuli'],
 ['titusi', 'asal', 'penuli'],
 ['nama', 'penulis', 'nama', 'penulis', 'dst'],
 ['afiliasi', 'penulis', 'ingtitusi', 'asal', 'penuli'],
 ['afiliasi', 'penulis', 'asal', 'penuli'],
 ['email', 'penuli', 'saja', 'bama'],
 ['abstrak'],
 ['jurnal',
  'online',
  'institut',
  'teknologi',
  'nasional',
  'bandung',
  'terdiri',
  'dari',
  'jurnal',
  'yang'],
 ['memiliki',
  'fokus',
  'pada',
  'bidang',
  'kajian',
  'ilmiah',
  'untuk',
  'memudahkan',
  'prose'],
 ['penyuntingan',
  'para',
  'penuli',
  'disarankan',
  'untuk',
  'mengikuti',
  'petunjuk',
  'penulisan'],
 ['artikel',
  'ini',
  'secara',
  'keseluruhan',
  'format',
  'dan',
  'syle',
  'yang',
  'terdapat',
  'dalam',
  'file',
  'ini',
  'sudah'],
 ['sesuai',
  'dengan',
  'spesifikasi',
  'yang',
  'tertulis',
  'dalam',
  'petunjuk',
  'penulisan',
  'sehingga',
  'file',
  'ini'],
 ['dapat',
  'digunakan',
  'sebagai',

In [9]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams

[['san', 'artikel'],
 ['teknologi', 'na'],
 ['penulis', 'dst'],
 ['titusi', 'asal', 'penuli'],
 ['titusi', 'asal', 'penuli'],
 ['nama', 'penulis', 'nama', 'penulis', 'dst'],
 ['afiliasi', 'penulis', 'ingtitusi', 'asal', 'penuli'],
 ['afiliasi', 'penulis', 'asal', 'penuli'],
 ['email', 'penuli', 'saja', 'bama'],
 ['abstrak'],
 ['jurnal',
  'online',
  'institut',
  'teknologi',
  'nasional',
  'bandung',
  'terdiri',
  'dari',
  'jurnal',
  'yang'],
 ['memiliki',
  'fokus',
  'pada',
  'bidang',
  'kajian',
  'ilmiah',
  'untuk',
  'memudahkan',
  'prose'],
 ['penyuntingan',
  'para',
  'penuli',
  'disarankan',
  'untuk',
  'mengikuti',
  'petunjuk',
  'penulisan'],
 ['artikel',
  'ini',
  'secara',
  'keseluruhan',
  'format',
  'dan',
  'syle',
  'yang',
  'terdapat',
  'dalam',
  'file',
  'ini',
  'sudah'],
 ['sesuai',
  'dengan',
  'spesifikasi',
  'yang',
  'tertulis',
  'dalam',
  'petunjuk',
  'penulisan',
  'sehingga',
  'file',
  'ini'],
 ['dapat',
  'digunakan',
  'sebagai',

# Step 3, Build Model

In [10]:
#TF-IDF REMOVAL

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [12]:
test_doc = corpus[-1]

vector = lda_model[test_doc]

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector = Sort(vector)

print (new_vector)

[(0, 0.58569854), (9, 0.3175083), (4, 0.024536127), (7, 0.013259546), (8, 0.012445089), (1, 0.010791636), (6, 0.010601475)]


In [13]:
lda_model.save("models/test_model.model")

In [14]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [15]:
test_doc = corpus[-1]

vector = new_model[test_doc]

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector = Sort(vector)
print (new_vector)

[(0, 0.5858768), (9, 0.3173301), (4, 0.024536122), (7, 0.013259546), (8, 0.012445089), (1, 0.010791636), (6, 0.010601475)]


# Step 4, Visual the Data

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\Raqwan\AppData\Roaming\Python\Python310\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.223606  0.200451       1        1  26.098651
0      0.034368  0.315333       2        1  18.770227
8     -0.178411  0.011412       3        1   9.368225
7     -0.204739  0.175700       4        1   8.539564
1     -0.026431 -0.117566       5        1   8.134972
5     -0.011905  0.053241       6        1   7.001683
3      0.068448 -0.016983       7        1   6.979879
9     -0.259331 -0.202921       8        1   5.598721
2      0.080696 -0.305250       9        1   5.240180
6      0.273699 -0.113417      10        1   4.267897, topic_info=            Term       Freq      Total Category  logprob  loglift
407      endnote  19.000000  19.000000  Default  30.0000  30.0000
0        artikel  51.000000  51.000000  Default  29.0000  29.0000
19        jurnal  16.000000  16.000000  Default  28.0000  28.0000
235  menggunakan  12.000000  12.000000  Default  27.0000  27.0000
17          dari  34.000000  34.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
61      template   0.640659   2.693843  Topic10  -4.8826   1.7178
406     terakhir   0.313611   1.353628  Topic10  -5.5969   1.6917
41          file   0.620538   2.987787  Topic10  -4.9145   1.5823
43           ini   1.300654  10.269369  Topic10  -4.1744   1.0878
38      petunjuk   0.465316   4.656532  Topic10  -5.2023   0.8507

[442 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
15        2  0.774429  abstrak
15        9  0.258143  abstrak
339       4  0.389351      ada
339       6  0.389351      ada
54        1  0.583527   adalah
...     ...       ...      ...
23        4  0.114552     yang
23        5  0.068731     yang
23        6  0.068731     yang
23        7  0.045821     yang
596      10  0.679302   yousda

[456 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 9, 8, 2, 6, 4, 10, 3, 7])

In [17]:
# Simpan koordinat titik visualisasi

data_to_save = {
    "topic_coordinates": vis.topic_coordinates.to_dict(orient="split"),
    "topic_info": vis.topic_info.to_dict(orient="split"),
    "token_table": vis.token_table.to_dict(orient="split"),
    "topic_order": vis.topic_order
}

# Simpan data ke dalam file JSON

with open("outputs/visualisasi_lda.json", "w") as json_file:
    json.dump(data_to_save, json_file)